### The Problem: 
Visualise results of what-if scenarios to assess one-off student loan repayment options before indexation of 7.1%

Known varaibles:
- Current loan balance (with transaction records available if occurring/occurred in the past years)
- Age of the loan: partial effects if part of loan newer than 11 months
- Earnings financial year 2022-2023 (maybe incl. voluntary repayment made in the period)
- If have home loan with offset account: current home loan rate
- If saving for home loan deposit: desired borrowing power
- Other accounts with interest (e.g. saving, investment)

Variables that need to be estimated:
- Future indexation rates
- Future earnings => determines compulsory repayment amount
- Planned voluntary repayments
- If have home loan with offset account: future home loan rates

Hypothesis 1: Paying off is only beneficial if having high-paying jobs and will pay off the loan in the next 2-3 years naturally in any cases

Hypothesis 2: Partial pay off is a worse idea than paying off fully or only making minimum repayment for the rest of loan course (with an offset account)

In [1]:
import pandas as pd

In [ ]:
# indexation rate table 
# https://www.ato.gov.au/Rates/Study-and-training-loan-indexation-rates/
# data = {'Year': [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013],
#         'Indexation rate': ['7.1%', '3.9%', '0.6%', '1.8%', '1.8%', '1.9%', '1.5%', '1.5%', '2.1%', '2.6%', '2.0%']}

# df = pd.DataFrame(data)

In [2]:
# 2022 - 2023 repayment income thresholds and rates
# https://www.ato.gov.au/Rates/HELP,-TSL-and-SFSS-repayment-thresholds-and-rates/
repayment_rate_thre_data = [
    (48361, 0),
    (55836, 0.01),
    (59186, 0.02),
    (62738, 0.025),
    (66502, 0.03),
    (70492, 0.035),
    (74722, 0.04),
    (79206, 0.045),
    (83958, 0.05),
    (88996, 0.055),
    (94336, 0.06),
    (99996, 0.065),
    (105996, 0.07),
    (112355, 0.075),
    (119097, 0.08),
    (126243, 0.085),
    (133818, 0.09),
    (141847, 0.095),
]

# Create a DataFrame
columns = ["Income Threshold", "Repayment Rate"]
df_repayment_rate_thre = pd.DataFrame(repayment_rate_thre_data, columns=columns)

# Display the DataFrame
print(df_repayment_rate_thre)

    Income Threshold  Repayment Rate
0              48361           0.000
1              55836           0.010
2              59186           0.020
3              62738           0.025
4              66502           0.030
5              70492           0.035
6              74722           0.040
7              79206           0.045
8              83958           0.050
9              88996           0.055
10             94336           0.060
11             99996           0.065
12            105996           0.070
13            112355           0.075
14            119097           0.080
15            126243           0.085
16            133818           0.090
17            141847           0.095


In [3]:
# assume repayment income thresholds and rates stays the same for now 
def get_repayment_rate(income):
    rates = [
        (48361, 0),
        (55836, 0.01),
        (59186, 0.02),
        (62738, 0.025),
        (66502, 0.03),
        (70492, 0.035),
        (74722, 0.04),
        (79206, 0.045),
        (83958, 0.05),
        (88996, 0.055),
        (94336, 0.06),
        (99996, 0.065),
        (105996, 0.07),
        (112355, 0.075),
        (119097, 0.08),
        (126243, 0.085),
        (133818, 0.09),
        (141847, 0.095),
    ]

    for income_threshold, rate in rates:
        if income <= income_threshold:
            return rate
    return 0.1

income = 60000
repayment_rate = get_repayment_rate(income)
print(f"The repayment rate for an income of ${income} is {repayment_rate*100}%")

The repayment rate for an income of $60000 is 2.5%


In [22]:
def calc_compulsory_repayment(earnings):
    return earnings * get_repayment_rate(earnings) #*increased loan balance does not affect compulsory repayment - will just take longer

def apply_indexation(loan_balance, indexation_rate=0.025):
    return loan_balance * (1 + indexation_rate)

def estimate_future_earnings(initial_earnings, years, growth_rate=0.01):
    return initial_earnings * (1 + growth_rate) ** years

In [9]:
# year 0: 2023, assuming fixed/averaged home loan rate for each year, assumed rates from reddit
home_loan_rates = {0: 0.0582, 1: 0.0499, 2: 0.0499, 3: 0.0549, 4: 0.0599, 5:0.0599, 6:0.0599, 7:0.0599}
indexation_rates = {0: 0.071, 1: 0.05, 2: 0.04, 3: 0.03, 4:0.02, 5:0.02, 6:0.02, 7:0.02}

#https://www.reddit.com/r/fiaustralia/comments/1304agl/pay_off_hecs_or_leave_money_in_offset/

In [23]:
def calc_ongoing_loan_balance(initial_loan_balance, indexation_rates):
    loan_balance = initial_loan_balance
    loan_balances = [loan_balance]

    for year, indexation_rate in indexation_rates.items():
        loan_balance *= (1 + indexation_rate)
        loan_balances.append(loan_balance)

    return loan_balances

initial_loan_balance = 70000 
loan_balances = calc_ongoing_loan_balance(initial_loan_balance, indexation_rates)

print("Year\tLoan Balance")
for year, balance in enumerate(loan_balances):
    print(f"{year}\t{balance:.2f}")

Year	Loan Balance
0	70000.00
1	74970.00
2	78718.50
3	81867.24
4	84323.26
5	86009.72
6	87729.92
7	89484.52
8	91274.21


In [99]:
def calc_remaining_years_on_year_X(loan_balances, initial_earnings, growth_rate, year_X=0, voluntary_repayments=0, verbose=False):
    year = 0
    remaining_loan_balance = loan_balances[year_X]
    if verbose:
        print(f"loan_balance on year {year_X}: {remaining_loan_balance}")

    while remaining_loan_balance > 0:
        earnings = initial_earnings * (1 + growth_rate) ** year
        repayment = calc_compulsory_repayment(earnings)
        remaining_loan_balance -= repayment
        year += 1
        if verbose:
            print(f"remaining balance: {remaining_loan_balance} on year {year}")
    return year

initial_earnings = 68000  
growth_rate = 0.03  

remaining_years = calc_remaining_years_on_year_X(loan_balances, initial_earnings, growth_rate, verbose=False)

print(f"Remaining years to pay off the student loan on year 0: {remaining_years}")

Remaining years to pay off the student loan on year 0: 16


In [37]:
def simulate_yearly_gain_home_loan_case(amount, year, earnings, voluntary_repayments):
    """yearly gain from offset account"""
    
    # increased loan_balance due to indexation
    increased_loan_balance = loan_balances[year]-loan_balances[year-1]
    
    # cost from the student loan = compulsary repayment + any voluntary_repayment
    cost_loan = calc_compulsory_repayment(earnings) + voluntary_repayments
    
    # earnings from offset account - if the one_off_payment_amount all goes to the offset account
    earnings_offset = amount * home_loan_rate[year-1]
    
    yearly_gain = earnings_offset - cost_loan
    
    return yearly_gain

In [121]:
# # simulate over years
# def simulate_over_years(cash=60000, initial_loan_balance=70000, initial_earnings=68000, growth_rate=0.03, voluntary_repayments=0, years=5):
    
#     # check if amount of cash at hand is larger or smaller than current loan balance
#     if cash > initial_loan_balance:
#         max_amount = initial_loan_balance
#     else:
#         max_amount = cash
    
#     one_off_payment_range = range(0, max_amount, 10000) 

#     gains = []
#     remaining_years_lst = []
#     for year in range(1, years+1):

#         earnings = estimate_future_earnings(initial_earnings, year, growth_rate=growth_rate)
        
#         yearly_gain_data = []
#         remaining_years_data = []
        
#         for one_off_payment in one_off_payment_range:
#             amount = max_amount - one_off_payment
            
#             yearly_gain = simulate_yearly_gain_home_loan_case(amount, year, earnings, voluntary_repayments)
            
#             loan_balances = calc_ongoing_loan_balance(initial_loan_balance-one_off_payment, indexation_rates)
# #             print("Year\tLoan Balance")
# #             for y, b in enumerate(loan_balances):
# #                 print(f"{y}\t{b:.2f}")
#             remaining_years = calc_remaining_years_on_year_X(loan_balances, initial_earnings, growth_rate, year_X=year, voluntary_repayments=0)
#             print(one_off_payment, remaining_years)
#             yearly_gain_data.append(yearly_gain) # data of different one_off_payment amount
#             remaining_years_data.append(remaining_years)
            
#         gains.append(yearly_gain_data)
#         remaining_years_lst.append(remaining_years_data)
#     return gains, remaining_years_lst

In [83]:
# simulate_over_years(cash=60000, initial_loan_balance=70000, initial_earnings=68000, growth_rate=0.0, voluntary_repayments=0, years=5)

In [129]:
# simulate over years
def simulate_over_years(cash=60000, initial_loan_balance=70000, initial_earnings=68000, growth_rate=0.03, voluntary_repayments=0, years=5, print_estimated_loan_balances=False):
    
    # year input cannot be longer than estimated years of rates | also need to be shorter than maximum remaining years?
    
    # check if amount of cash at hand is larger or smaller than current loan balance
    if cash > initial_loan_balance:
        max_amount = initial_loan_balance
    else:
        max_amount = cash
    
    one_off_payment_range = range(0, max_amount, 10000) 
    
            
    gains = []
    remaining_years_lst = []
    
    for one_off_payment in one_off_payment_range:
        
        amount = max_amount - one_off_payment
        loan_balances = calc_ongoing_loan_balance(initial_loan_balance-one_off_payment, indexation_rates)
        if print_estimated_loan_balances:
            print("Year\tLoan Balance")
            for y, b in enumerate(loan_balances):
                print(f"{y}\t{b:.2f}")
        
        gains_data = []
        remaining_years_data = []
        
        for year in range(1, years+1):
            earnings = estimate_future_earnings(initial_earnings, year, growth_rate=growth_rate)
            yearly_gain = simulate_yearly_gain_home_loan_case(amount, year, earnings, voluntary_repayments)
            remaining_years = calc_remaining_years_on_year_X(loan_balances, initial_earnings, growth_rate, year_X=year, voluntary_repayments=0)
        
            gains_data.append(yearly_gain)
            remaining_years_data.append(remaining_years)
        
        gains.append(gains_data)
        remaining_years_lst.append(remaining_years_data)
        
        print(f"With one-off payment of {one_off_payment} on year 0, the remaining years to pay off the student loan on year 1 is {remaining_years_data[0]}")
        
    return gains, remaining_years_lst

In [130]:
simulate_over_years(cash=60000, initial_loan_balance=70000, initial_earnings=68000, growth_rate=0.03, voluntary_repayments=0, years=5)

With one-off payment of 0 on year 0, the remaining years to pay off the student loan on year 1 is 17
With one-off payment of 10000 on year 0, the remaining years to pay off the student loan on year 1 is 15
With one-off payment of 20000 on year 0, the remaining years to pay off the student loan on year 1 is 14
With one-off payment of 30000 on year 0, the remaining years to pay off the student loan on year 1 is 12
With one-off payment of 40000 on year 0, the remaining years to pay off the student loan on year 1 is 10
With one-off payment of 50000 on year 0, the remaining years to pay off the student loan on year 1 is 7


([[1040.6,
   108.35199999999986,
   21.782560000000103,
   -150.0569586000006,
   46.62133264199974],
  [458.5999999999999,
   -390.64800000000014,
   -477.2174399999999,
   -699.0569586000006,
   -552.3786673580003],
  [-123.40000000000009,
   -889.6480000000001,
   -976.2174399999999,
   -1248.0569586000006,
   -1151.3786673580003],
  [-705.4000000000001,
   -1388.6480000000001,
   -1475.21744,
   -1797.0569586000006,
   -1750.3786673580003],
  [-1287.4,
   -1887.6480000000001,
   -1974.21744,
   -2346.0569586000006,
   -2349.3786673580003],
  [-1869.4, -2386.648, -2473.21744, -2895.0569586000006, -2948.3786673580003]],
 [[17, 17, 17, 18, 18],
  [15, 16, 16, 16, 16],
  [14, 14, 14, 15, 15],
  [12, 12, 13, 13, 13],
  [10, 10, 10, 11, 11],
  [7, 8, 8, 8, 8]])

In [146]:
one_off_payment_range = range(0, 60000, 10000) 
gain_or_loss_data, _ = simulate_over_years(cash=60000, initial_loan_balance=70000, initial_earnings=68000, growth_rate=0.03, voluntary_repayments=0, years=3)

With one-off payment of 0 on year 0, the remaining years to pay off the student loan on year 1 is 17
With one-off payment of 10000 on year 0, the remaining years to pay off the student loan on year 1 is 15
With one-off payment of 20000 on year 0, the remaining years to pay off the student loan on year 1 is 14
With one-off payment of 30000 on year 0, the remaining years to pay off the student loan on year 1 is 12
With one-off payment of 40000 on year 0, the remaining years to pay off the student loan on year 1 is 10
With one-off payment of 50000 on year 0, the remaining years to pay off the student loan on year 1 is 7


In [179]:
import plotly.graph_objs as go

def plot_area_gain_or_loss(one_off_payment_range, gain_or_loss_data, title):
    fig = go.Figure()
    color_palette = ['#cb8b94', '#ac9a78', '#7ea677', '#7ca4a0', '#819fc1', '#be8bc4']
    for i, one_off_payment in enumerate(one_off_payment_range):
        fig.add_trace(go.Scatter(x=list(range(1, len(gain_or_loss_data[0]) + 1)), y=gain_or_loss_data[i], 
                                 stackgroup='one', name=f'One-Off Payment: {one_off_payment}', 
                                 line=dict(color=color_palette[i % len(color_palette)])))
    fig.update_layout(title=title, xaxis_title="Years", yaxis_title="Gain or Loss",
#                         title=title,
#                         plot_bgcolor="#FFF",  # Sets background color to white
#                         xaxis=dict(
#                             title="Years",
#                             linecolor="#BCCCDC",  # Sets color of X-axis line
#                             showgrid=True  # Removes X-axis grid lines
#                         ),
#                         yaxis=dict(
#                             title="Gain or Loss",  
#                             linecolor="#BCCCDC",  # Sets color of Y-axis line
#                             #showgrid=False,  # Removes Y-axis grid lines    
#                         )
)

    fig.show()

plot_area_gain_or_loss(one_off_payment_range, gain_or_loss_data, "Gain or Loss for Different One-Off Payment Amounts Over Time")

In [171]:
import seaborn as sns
sns.husl_palette(s=.4)

[(0.7973774215435722, 0.5437247724897194, 0.5792120727646433),
 (0.6745106399346137, 0.6038862582260649, 0.47068885861899207),
 (0.4954406045983249, 0.6526830324326673, 0.46857428049966526),
 (0.4859798659523616, 0.6418250670095154, 0.6259446925861523),
 (0.5047731287519484, 0.6229130595802063, 0.7566756173582012),
 (0.7440150453033102, 0.5433228014843097, 0.7689691084933888)]

In [172]:
print(sns.husl_palette(s=.4).as_hex())

['#cb8b94', '#ac9a78', '#7ea677', '#7ca4a0', '#819fc1', '#be8bc4']
